In [29]:
import os
import pandas as pd
import getpass
from langchain_core.prompts import ChatPromptTemplate
from langchain_experimental.utilities import PythonREPL
from pydantic import BaseModel, Field  # Ensure compatibility with Pydantic v2
from langchain.agents import Tool
from langchain_cohere.chat_models import ChatCohere
from langchain_cohere.react_multi_hop.agent import create_cohere_react_agent
from langchain.agents import AgentExecutor

# Ensure the Cohere API Key is set securely
os.environ["COHERE_API_KEY"] = getpass.getpass("Enter your Cohere API Key: ")

# Define the chat prompt template
prompt = ChatPromptTemplate.from_template("{input}")

# Define the Python execution tool
python_repl = PythonREPL()
python_tool = Tool(
    name="python_interpreter",
    description="Executes Python code and returns the result. The code runs in a static sandbox without interactive mode, so print output or save output to a file.",
    func=python_repl.run,
)

# Define the input schema for the Python tool
class ToolInput(BaseModel):
    code: str = Field(description="Python code to execute.")

# Assign schema to the tool (Ensuring compatibility)
python_tool.args_schema = ToolInput  # Pydantic v2

# Define available tools
tools = [python_tool]

# Define the Cohere language model
llm = ChatCohere(model="command-r", temperature=0)

# Instantiate the agent and executor
agent = create_cohere_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt,
)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, return_intermediate_steps=True)

# Sample question: Generate Python code 
q1 = "write a python code for array sum operation with recursion?"
a1 = agent_executor.invoke({"input": q1})

print("Generated Code:\n", a1)


Enter your Cohere API Key:  ········




> Entering new AgentExecutor chain...

I will write and execute Python code that uses recursion to perform an operation on an array.
{'tool_name': 'python_interpreter', 'parameters': {'code': 'def sum_array(arr):\n    # Base case\n    if len(arr) == 0:\n        return 0\n\n    # Recursive case\n    return arr[0] + sum_array(arr[1:])\n\n# Test the function\narray = [1, 2, 3, 4, 5]\nresult = sum_array(array)\nprint(result)\n'}}
1;3mNameError("name 'sum_array' is not defined")
My code has produced a NameError, indicating that the function 'sum_array' is not defined. I will fix this issue by adding the appropriate function definition.
{'tool_name': 'python_interpreter', 'parameters': {'code': 'def sum_array(arr):\n    return sum_array_helper(arr, 0)\n\ndef sum_array_helper(arr, index):\n    # Base case\n    if index == len(arr):\n        return 0\n\n    # Recursive case\n    return arr[index] + sum_array_helper(arr, index + 1)\n\n# Test the function\narray = [1, 2, 3, 4, 5]\nresult = sum